# OpenAI Agents

In [1]:
!pip install -Uq openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 6.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 14.1 MB/s eta 0:00:00


In [24]:
# to use non-openai models, e.g., from Hugging Face
!pip install -Uq "openai-agents[litellm]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 90.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 119.2 MB/s eta 0:00:0000:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires grpcio>=1.71.2, but you have grpcio 1.67.1 which is incompatible.


In [ ]:
# enable async in notebook
import nest_asyncio

nest_asyncio.apply()

In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [23]:
# set default model for agents

os.environ["OPENAI_DEFAULT_MODEL"] = "gpt-5-mini"

## Quickstart

In [ ]:
from agents import Agent, function_tool

@function_tool
def get_weather(city: str) -> str:
    """returns weather info for the specified city."""
    return f"The weather in {city} is sunny"

agent = Agent(
    name="Haiku agent",
    instructions="Always respond in haiku form",
    model="gpt-5-mini",
    tools=[get_weather],
)

In [ ]:
from agents import Runner

result = await Runner.run(agent, "What's the weather in New York?")

In [22]:
print(type(result))

from pprint import pprint
pprint(result.__dict__, indent=1)

<class 'agents.result.RunResult'>
{'_last_agent': Agent(name='Haiku agent',
                      handoff_description=None,
                      tools=[FunctionTool(name='get_weather',
                                          description='returns weather info '
                                                      'for the specified city.',
                                          params_json_schema={'additionalProperties': False,
                                                              'properties': {'city': {'title': 'City',
                                                                                      'type': 'string'}},
                                                              'required': ['city'],
                                                              'title': 'get_weather_args',
                                                              'type': 'object'},
                                          on_invoke_tool=<function function_tool.<locals>._create

In [18]:
print(result.final_output)

Sun warms city streets  
clear blue sky, no cloud in sight  
Smiles bloom on faces


## Non Open-AI models

Use Hugging Face models with OpenAI Agents SDK.

Just add the HF_TOKEN env variable. And set the model param to:

```
litellm/huggingface/<provider>/<hf_org_or_user>/<hf_model>
```

In [31]:
import getpass

os.environ["HF_TOKEN"] = getpass.getpass("Enter your Hugging Face token: ")

In [ ]:
# using kimi-k2-thinking

from agents import Agent, Runner, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel

agent_kimi = Agent(
    name="Kimi agent",
    instructions="Always respond in haiku form",
    # model="litellm/huggingface/novita/MiniMaxAI/MiniMax-M2.1",
    tools=[get_weather],
    
    # optional, for tracing (requires openai API key)
    model=LitellmModel(model="huggingface/novita/MiniMaxAI/MiniMax-M2.1", api_key=os.environ["HF_TOKEN"]),
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(agent_kimi, "What's the weather in New York?")

In [41]:
pprint(result.__dict__)

{'_last_agent': Agent(name='Kimi agent',
                      handoff_description=None,
                      tools=[FunctionTool(name='get_weather',
                                          description='returns weather info '
                                                      'for the specified city.',
                                          params_json_schema={'additionalProperties': False,
                                                              'properties': {'city': {'title': 'City',
                                                                                      'type': 'string'}},
                                                              'required': ['city'],
                                                              'title': 'get_weather_args',
                                                              'type': 'object'},
                                          on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_

In [42]:
print(result.final_output)


Sunny New York day
Golden light fills the streets
Summer warmth embraces


## Agents

### Structured Output

- Use the `output_type` param with Pydantic objects (or dataclasses, lists, TypedDicts, etc.)
- When using non-openai models, remember to check that they support both Structured Ouput AND tool calling.

In [ ]:
from pydantic import BaseModel
from agents import Agent


class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

model = LitellmModel(
    model="huggingface/novita/zai-org/GLM-4.7",
    api_key=os.environ["HF_TOKEN"],
)

agent = Agent(
    name="Calendar extractor",
    instructions="Extract calendar events from text",
    output_type=CalendarEvent,
    model=model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    agent,
    "Extract the event from the following text: 'Meeting with Alice and Bob on July 5th.'",
)

print(result.final_output)

name='Meeting with Alice and Bob' date='July 5th' participants=['Alice', 'Bob']


### Multi-Agent systems

Two main architectures:
- Manager (agents as tools): A central manager/orchestrator invokes specialized sub‑agents as tools and retains control of the conversation.
- Handoffs: Peer agents hand off control to a specialized agent that takes over the conversation. This is decentralized.


#### Agents Handoffs

In [47]:
from agents import Agent

glm_model = LitellmModel(
    model="huggingface/novita/zai-org/GLM-4.7",
    api_key=os.environ["HF_TOKEN"],
)

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    triage_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

# Causes of World War II

World War II (1939-1945) emerged from a complex combination of political, economic, and social factors. Here are the key causes:

## 1. Treaty of Versailles (1919)
The peace settlement ending World War I imposed harsh penalties on Germany, including:
- Massive reparations payments
- Territorial losses
- Military restrictions
- War guilt clause assigning full responsibility to Germany

These provisions created deep resentment in Germany and economic hardship that fueled extremist politics.

## 2. Rise of Totalitarian Regimes
- **Germany:** Adolf Hitler and the Nazi Party came to power in 1933, promoting German nationalism, anti-Semitism, and territorial expansion.
- **Italy:** Benito Mussolini established a fascist dictatorship in the 1920s with imperial ambitions.
- **Japan:** Military leaders gained control, pursuing aggressive expansion in Asia.

## 3. Expansionist Aggression
- **Germany:** Remilitarized the Rhineland (1936), annexed Austria (1938), and dema

#### Agents As Tools

In [49]:
manager_agent = Agent(
    name="Manager Agent",
    instructions="You manage a team of agents to answer user questions effectively.",
    tools=[
        history_tutor_agent.as_tool(
            tool_name="history_tutor_agent",
            tool_description="Handles historical queries",
        ),
        math_tutor_agent.as_tool(
            tool_name="math_tutor_agent",
            tool_description="Handles math questions",
        ),
    ],
    model=glm_model,
    model_settings=ModelSettings(include_usage=True),
)

result = await Runner.run(
    manager_agent,
    "Can you explain the causes of World War II?",
)

print(result.final_output)

The causes of World War II were complex and rooted in unresolved issues from World War I, economic instability, and the rise of aggressive totalitarian regimes. Here's a breakdown:

## 1. The Treaty of Versailles (1919)
- Imposed harsh penalties on Germany: accepted full responsibility, paid massive reparations, lost territory, faced military restrictions
- Created deep resentment and humiliation, fueling radical political movements

## 2. Rise of Totalitarianism and Fascism
- **Adolf Hitler (Germany):** Promoted German superiority (*Lebensraum*) and sought to overturn the Treaty
- **Benito Mussolini (Italy):** Pursued imperial ambitions in Africa and the Mediterranean
- **Military Japan:** Sought domination of East Asia to secure resources

## 3. Failure of the League of Nations
- Lacked enforcement power and key member support
- Weak response to Japan's invasion of Manchuria (1931) and Italy's invasion of Ethiopia (1935)
- Emboldened aggressors by showing they could act without conse